<a href="https://colab.research.google.com/github/mariliaoliv/Computacao-Evolutiva/blob/main/eightqueens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Melhor Configuração Geral:**

Pop=150, Mut=0.15: 87.5% de sucesso em apenas 147.5 gerações (0.66s)

Pop=200, Mut=0.01/0.1: 87.5% de sucesso com ~180 gerações (~0.7s)

**Tendências Observadas:**

Populações maiores (150-200) geralmente performam melhor

Taxas de mutação médias (0.05-0.15) são mais eficazes

O pior caso foi Pop=50, Mut=0.01/0.15 (apenas 25% de sucesso)

O que piora:

Populações muito pequenas (50)

Taxas de mutação extremas (0.01 ou 0.15 para Pop>150)

**Com base nos testes:**

Para eficiência: Pop=150, Mut=0.15 (melhor equilíbrio)

Para confiabilidade: Pop=200, Mut=0.1 (resultado consistente)

Para velocidade: Pop=100, Mut=0.05 (75% sucesso em 0.47s)

In [ ]:
import random
import time
from collections import defaultdict

# Meu
# Configurações
POPULATION_SIZES = [50, 100, 150, 200]
MUTATION_RATES = [0.01, 0.05, 0.08, 0.1, 0.15]
N_RUNS = 8
MAX_GENERATIONS = 1000
TARGET_FITNESS = 28

# Funções auxiliares
def create_individual():
    return [random.randint(0, 7) for _ in range(8)]

def evaluate_fitness(genes):
    clashes = 0
    for i in range(8):
        for j in range(i + 1, 8):
            if genes[i] == genes[j] or abs(genes[i] - genes[j]) == abs(i - j):
                clashes += 1
    return 28 - clashes

def create_population(pop_size):
    return sorted([(create_individual(), evaluate_fitness(create_individual()))
                  for _ in range(pop_size)], key=lambda x: x[1], reverse=True)

def select_parent(population, tournament_size=5):
    candidates = random.sample(population, tournament_size)
    return max(candidates, key=lambda x: x[1])

def crossover(parent1, parent2):
    point1, point2 = sorted(random.sample(range(1, 8), 2))
    child = parent1[0][:point1] + parent2[0][point1:point2] + parent1[0][point2:]
    return (child, evaluate_fitness(child))

def adaptive_mutation(individual, mutation_rate, generation, last_improvement):
    if generation - last_improvement > 50:
        mutation_rate = min(0.3, mutation_rate * 1.2)
    genes, fitness = individual
    if random.random() < mutation_rate:
        genes[random.randint(0, 7)] = random.randint(0, 7)
    return (genes, evaluate_fitness(genes)), mutation_rate

def evolve_population(population, mutation_rate, generation, last_improvement):
    new_pop = population[:20]
    for _ in range(len(population) - 20):
        parent1 = select_parent(population)
        parent2 = select_parent(population)
        child = crossover(parent1, parent2)
        child, mutation_rate = adaptive_mutation(child, mutation_rate, generation, last_improvement)
        new_pop.append(child)
    new_pop.sort(key=lambda x: x[1], reverse=True)
    return new_pop, mutation_rate

# Teste automatizado
def run_optimized_test():
    results = defaultdict(list)
    for pop_size in POPULATION_SIZES:
        for mut_rate in MUTATION_RATES:
            print(f"\n► Testando Pop={pop_size}, Mut={mut_rate}...")
            success_count = 0
            for _ in range(N_RUNS):
                pop = create_population(pop_size)
                best_fitness = pop[0][1]
                last_improvement = 0
                current_mut_rate = mut_rate
                start_time = time.time()
                for generation in range(MAX_GENERATIONS):
                    if best_fitness == TARGET_FITNESS:
                        success_count += 1
                        break
                    pop, current_mut_rate = evolve_population(pop, current_mut_rate, generation, last_improvement)
                    new_best = pop[0][1]
                    if new_best > best_fitness:
                        best_fitness = new_best
                        last_improvement = generation
                exec_time = time.time() - start_time
                results[(pop_size, mut_rate)].append((generation, exec_time, best_fitness))
            success_rate = (success_count / N_RUNS) * 100
            avg_gens = sum(r[0] for r in results[(pop_size, mut_rate)]) / N_RUNS
            avg_time = sum(r[1] for r in results[(pop_size, mut_rate)]) / N_RUNS
            print(f"  ✓ Sucesso: {success_rate}% | Gerações: {avg_gens:.1f} | Tempo: {avg_time:.2f}s")
    return results

if __name__ == "__main__":
    final_results = run_optimized_test()




► Testando Pop=50, Mut=0.01...
  ✓ Sucesso: 25.0% | Gerações: 773.9 | Tempo: 0.51s

► Testando Pop=50, Mut=0.05...
  ✓ Sucesso: 37.5% | Gerações: 738.9 | Tempo: 0.48s

► Testando Pop=50, Mut=0.08...
  ✓ Sucesso: 50.0% | Gerações: 595.9 | Tempo: 0.48s

► Testando Pop=50, Mut=0.1...
  ✓ Sucesso: 12.5% | Gerações: 892.8 | Tempo: 0.64s

► Testando Pop=50, Mut=0.15...
  ✓ Sucesso: 50.0% | Gerações: 550.4 | Tempo: 0.36s

► Testando Pop=100, Mut=0.01...
  ✓ Sucesso: 50.0% | Gerações: 612.4 | Tempo: 1.21s

► Testando Pop=100, Mut=0.05...
